In [ ]:
import geopyspark as gps
import numpy as np
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from geonotebook.wrappers import TMSRasterData

conf = gps.geopyspark_conf(appName="gps-osm-ingest", master='local[8]')
conf.set("spark.hadoop.yarn.timeline-service.enabled", False)
conf.set('spark.ui.enabled', True)
conf.set('spark.default.parallelism', 8)
conf.set('spark.master.memory', '9500M')

sc = SparkContext(conf=conf)
pysc = gps.get_spark_context()
session = SparkSession.builder.config(conf=pysc.getConf()).enableHiveSupport().getOrCreate()

In [ ]:
file_uri = "file:/Users/eugene/Downloads/isle-of-man.orc"
osm_dataframe = session.read.orc(file_uri)

In [ ]:
osm = gps.vector_pipe.osm_reader.from_dataframe(osm_dataframe)
lines = osm.get_line_features_rdd()

In [ ]:
highways = lines.filter(
    lambda feature: 'highway' in feature.properties.tags)

In [ ]:
highway_features = highways.map(
    lambda feature: gps.Feature(feature.geometry, gps.CellValue(2, 1)))

In [ ]:
highway_raster = gps.geotrellis.rasterize_features(
    features = highway_features,
    crs = 4326,
    zoom = 10,
    cell_type=gps.CellType.INT8RAW,
    num_partitions = 16
).persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
# insert block that renders the lines

In [ ]:
color_map = gps.ColorMap.from_colors(
    breaks = np.arange(0, 3, 1), 
    color_list = gps.get_colors_from_matplotlib('magma'))

osm_wm = highway_raster.tile_to_layout(
    gps.GlobalLayout(), target_crs=3857)
layer = gps.TMS.build(osm_wm.pyramid(), color_map)

M.add_layer(TMSRasterData(layer), name="OSM")

In [ ]:
isle_extent = gps.Extent(xmin=-4.921879863281248, ymin=53.964837753906245, 
                         xmax=-4.218754882812476, ymax=54.4921814453125)
#isle_extent = highway_raster.layer_metadata.extent

# Read SRTM for the same extent as rasterized OSM features
srtm = gps.query(
    uri="s3://geotrellis-test/dg-srtm",
    layer_name="srtm-wsg84-gps", 
    layer_zoom = 0,
    query_geom = isle_extent,
    query_proj = 4326,
    num_partitions = 16
)

# Tile SRTM layer to same layout as rasterized OSM features
tiled_srtm = srtm.tile_to_layout(highway_raster.layer_metadata)

In [ ]:
# Compensate for cell size at 55N
z_factor = 0.000015935

# TODO: GeoPySpark has smart slope function now where it constructs z-factor
tobler_raster = tiled_srtm.\
    focal(gps.Operation.SLOPE, param_1=z_factor).\
    tobler()
    
friction = tobler_raster - highway_raster

In [ ]:
reprojected = friction.tile_to_layout(
    target_crs = 3857,
    layout = gps.GlobalLayout(tile_size=256),
    resample_method = gps.ResampleMethod.MAX
).persist(StorageLevel.MEMORY_AND_DISK_SER)

In [ ]:
pyramid = reprojected.pyramid().persist(StorageLevel.MEMORY_AND_DISK_SER)

In [ ]:
color_map = gps.ColorMap.from_colors(
    breaks = np.arange(0, 7.5, 0.2), 
    color_list = gps.get_colors_from_matplotlib('magma'))

layer = gps.TMS.build(pyramid, color_map)

M.add_layer(TMSRasterData(layer), name="ToblerOSM")

In [ ]:
pyramid.get_histogram().min_max()

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
reprojected.unpersist()

In [ ]:
# Write pyramid to the catalog
for layer in pyrmaid.levels.values():
    gps.write("s3://geotrellis-test/dg-osm-test", "gps-osm-ingest", layer)